In [119]:
import requests

In [120]:
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

## Open Geojson

In [121]:
bulgaria_gdf = gpd.read_file("data/bg.json")

In [122]:
bulgaria_gdf["name"] = bulgaria_gdf["name"].str.lower()
bulgaria_gdf = bulgaria_gdf.replace("grad sofiya", "sofia cap.")

## Get Economic data

In [123]:
r = requests.get("https://www.nsi.bg/en/content/3930/statistical-regions-district")

In [124]:
soup = BeautifulSoup(r.text, "html.parser")

In [125]:
trs = soup.find("table").find_all("tr")[2:-1]

In [126]:
districts = []
for i, tr in enumerate(trs):
    tds = tr.find_all("td")
    region = tds[0].text.replace("\xa0", "").lower().strip()
    if region[0] != "-":
        print(region)
        continue
    else:
        region = region[1:]
    total = round(
        sum([int(td.text.replace("\xa0", "")) for td in tr.find_all("td")[1:]]) / 12.0,
        2,
    )
    districts.append({"name": region, "wage": total})

severozapaden
severen tsentralen
severoiztochen
yugoiztochen
yugozapaden
yuzhen tsentralen


In [127]:
wage_df = pd.DataFrame(districts)

## Merge data and export

In [130]:
wage_gdf = bulgaria_gdf.merge(wage_df, on="name", how="left")

In [133]:
wage_gdf.sort_values("wage", ascending=False)

,source,id,name,geometry,wage
27,https://simplemaps.com,BG22,sofia cap.,"POLYGON ((687124.095 4711394.223, 685557.755 4...",3138.00
17,https://simplemaps.com,BG03,varna,"POLYGON ((1062579.525 4767018.77, 1062006.001 ...",2132.50
12,https://simplemaps.com,BG06,vratsa,"POLYGON ((742461.449 4771767.65, 741043.166 47...",2090.25
5,https://simplemaps.com,BG23,sofia,"POLYGON ((668657.249 4730654.088, 667341.507 4...",2059.83
19,https://simplemaps.com,BG24,stara zagora,"POLYGON ((923874.402 4679966.627, 924209.142 4...",2001.50
18,https://simplemaps.com,BG16,plovdiv,"POLYGON ((857897.31 4672679.569, 856115.661 46...",1965.58
10,https://simplemaps.com,BG18,ruse,"POLYGON ((927141.312 4891814.817, 927387.822 4...",1886.25
24,https://simplemaps.com,BG07,gabrovo,"POLYGON ((830984.799 4788980.869, 831584.609 4...",1839.92
0,https://simplemaps.com,BG02,burgas,"POLYGON ((1000317.041 4676714.84, 1000314.567 ...",1809.33
21,https://simplemaps.com,BG27,shumen,"POLYGON ((999200.386 4773868.829, 995485.95 47...",1808.92


In [134]:
wage_gdf = wage_gdf.to_crs(7803)
wage_gdf.to_file("data/bulgaria_avg_monthly_wage.gpkg")